## Mengambil data dari Google Earth Engine

In [1]:
import ee

# Authenticate to Earth Engine
ee.Authenticate()
ee.Initialize(project='602670742321')

In [2]:
# Mount GDRIVE
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/'

Mounted at /content/drive


In [3]:
import pandas as pd

# Tentukan wilayah
region_of_interest = ee.Geometry.Rectangle([110.0089, -8.2414, 110.8603, -7.4959]).buffer(5000)

# Tentukan rentang waktu
start_date = '2023-01-01'
end_date   = '2023-12-31'

# Dapatkan citra Sentinel-2 dan hitung NDVI
image = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(region_of_interest) \
    .filterDate(start_date, end_date) \
    .median()

# Hitung NDVI dari citra Sentinel-2
red_band = image.select('B4')  # Pilih band merah (Red)
nir_band = image.select('B8')  # Pilih band inframerah dekat (Near Infrared)
ndvi = nir_band.subtract(red_band).divide(nir_band.add(red_band)).rename('NDVI')

# Tambahkan NDVI ke citra
ndvi_image = image.addBands(ndvi)

ndvi_data = ndvi_image.select(['B4', 'B8', 'NDVI']).reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=region_of_interest,
    scale=1000,
)

# Konversi data ke Pandas DataFrame
df = pd.DataFrame.from_dict(ndvi_data.getInfo())

# Tentukan fungsi untuk menetapkan label berdasarkan rentang NDVI
def determine_label(ndvi_value):
    if ndvi_value < 0:
        return 'air'
    elif ndvi_value >= 0 and ndvi_value <= 0.2:
        return 'lahan terbangun'
    else:
        return 'vegetasi'

# Terapkan fungsi untuk menentukan label pada setiap baris DataFrame
df['label'] = df['NDVI'].apply(determine_label)
print(df.label.value_counts())

label
vegetasi           5648
air                2441
lahan terbangun    1654
Name: count, dtype: int64


In [6]:
df['tahun'] = '2023'
df.to_csv(PATH + 'data_2023.csv', index=False)
df

,B4,B8,NDVI,label,tahun
0,1788.700000,1649.500000,-0.040486,air,2023
1,1943.416667,1703.666667,-0.065737,air,2023
2,1954.333333,1781.000000,-0.046404,air,2023
3,1845.500000,1700.500000,-0.040891,air,2023
4,1897.000000,1745.750000,-0.041521,air,2023
...,...,...,...,...,...
9738,2308.500000,3638.500000,0.223642,vegetasi,2023
9739,2170.083333,3408.333333,0.221972,vegetasi,2023
9740,2262.666667,3647.416667,0.234303,vegetasi,2023
9741,2257.000000,3505.000000,0.216591,vegetasi,2023


Jalankan kode dibawah ini ketika sudah mengunduh data dari 2015-2023

In [7]:
df1 = pd.read_csv(PATH + 'data_2015.csv')
df2 = pd.read_csv(PATH + 'data_2016.csv')
df3 = pd.read_csv(PATH + 'data_2017.csv')
df4 = pd.read_csv(PATH + 'data_2018.csv')
df5 = pd.read_csv(PATH + 'data_2019.csv')
df6 = pd.read_csv(PATH + 'data_2020.csv')
df7 = pd.read_csv(PATH + 'data_2021.csv')
df8 = pd.read_csv(PATH + 'data_2022.csv')
df9 = pd.read_csv(PATH + 'data_2023.csv')
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
df.to_csv(PATH + 'data.csv', index=False)
df

,B4,B8,NDVI,label,tahun
0,639.000000,456.000000,-0.167123,air,2015
1,684.000000,494.000000,-0.161290,air,2015
2,789.000000,599.000000,-0.136888,air,2015
3,1280.000000,1061.000000,-0.093550,air,2015
4,1279.000000,1061.000000,-0.093162,air,2015
...,...,...,...,...,...
87682,2308.500000,3638.500000,0.223642,vegetasi,2023
87683,2170.083333,3408.333333,0.221972,vegetasi,2023
87684,2262.666667,3647.416667,0.234303,vegetasi,2023
87685,2257.000000,3505.000000,0.216591,vegetasi,2023
